In [ ]:
# Try out making an index for each readii image type
from pathlib import Path
import pandas as pd
from damply import dirs
from readii.io.loaders.general import loadImageDatasetConfig
from readii.process.config import get_full_data_name
from readii.io.writers.nifti_writer import NIFTIWriter

/home/bhkuser/bhklab/katy/readii_2_roqc/.pixi/envs/default/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = 'NSCLC-Radiomics_test'
# Load in dataset configuration settings from provided file
config_dir_path = dirs.CONFIG / 'datasets'
dataset_config = loadImageDatasetConfig(dataset, config_dir_path)
full_data_name = get_full_data_name(config_dir_path / dataset)

image_modality = dataset_config['MIT']['MODALITIES']['image']
mask_modality = dataset_config['MIT']['MODALITIES']['mask']

region = 'full'
permutation = 'shuffled'

match permutation:
    case 'none':
        tool_outputs = 'mit'
        image_glob_pattern = f'{image_modality}.nii.gz'

    case 'shuffled' | 'sampled' | 'randomized':
        tool_outputs = 'readii'
        image_glob_pattern = f'{image_modality}_{permutation}_{region}.nii.gz'
    case _:
        print(f'Unknown permutation type: {permutation}')
        raise ValueError(f'Unknown permutation type: {permutation}')

image_directory = dirs.PROCDATA / full_data_name / 'images' / f"{tool_outputs}_{dataset_config['DATASET_NAME']}"

image_files = sorted(image_directory.rglob(pattern = image_glob_pattern))

mask_directory = dirs.PROCDATA / full_data_name / 'images' / f"mit_{dataset_config['DATASET_NAME']}"
mask_files = sorted(mask_directory.rglob(pattern = f'{mask_modality}*/*.nii.gz'))

In [35]:
print(image_files)
print(mask_files)

[PosixPath('/home/bhkuser/bhklab/katy/readii_2_roqc/data/procdata/TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-Radiomics_test/LUNG1-001_0000/CT_63382046/GTV/CT_shuffled_full.nii.gz'), PosixPath('/home/bhkuser/bhklab/katy/readii_2_roqc/data/procdata/TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-Radiomics_test/LUNG1-002_0001/CT_23261228/GTV/CT_shuffled_full.nii.gz')]
[PosixPath('/home/bhkuser/bhklab/katy/readii_2_roqc/data/procdata/TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Radiomics_test/LUNG1-001_0000/RTSTRUCT_35578236/GTV.nii.gz'), PosixPath('/home/bhkuser/bhklab/katy/readii_2_roqc/data/procdata/TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Radiomics_test/LUNG1-002_0001/RTSTRUCT_43245931/GTV.nii.gz')]


In [ ]:
mask_index = pd.DataFrame(data = {'ID': [mask_path.parent.parent.stem for mask_path in mask_files],
                                  'Mask': mask_files})
image_index = pd.DataFrame()

## Construct from MIT index

In [3]:
mit_simple_index = pd.read_csv(dirs.PROCDATA / full_data_name / 'images' / f'mit_{dataset_config["DATASET_NAME"]}' / f'mit_{dataset_config["DATASET_NAME"]}_index-simple.csv')

In [4]:
mit_simple_index

,filepath,hash,saved_time,SampleNumber,ImageID,PatientID,Modality,SeriesInstanceUID,StudyInstanceUID,ReferencedSeriesUID,...,direction,bbox.size,bbox.min_coord,bbox.max_coord,sum,min,max,mean,std,variance
0,LUNG1-001_0000/CT_63382046/CT.nii.gz,b4175e0d6ddff17507e30c05c6e7302c2b9668f4,2025-06-09:20:04:28,0,CT,LUNG1-001,CT,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,NaN,...,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)",NaN,NaN,NaN,-2.604295e+10,-1024.0,3034.0,-741.387913,426.958598,182293.644053
1,LUNG1-001_0000/RTSTRUCT_35578236/GTV.nii.gz,7afa5f960c8113372e0ad0a8e8303f7eaea396f4,2025-06-09:20:04:33,0,GTV,LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)","(98, 91, 21)","(290, 226, 65)","(388, 317, 86)",5.627100e+04,0.0,1.0,0.001602,0.039992,0.001599
2,LUNG1-002_0001/CT_23261228/CT.nii.gz,e18ddb88c4ca54b93c1d28d7db625ed5a9361664,2025-06-09:20:04:26,1,CT,LUNG1-002,CT,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,NaN,...,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)",NaN,NaN,NaN,-2.197104e+10,-1024.0,3071.0,-755.070802,431.016943,185775.604855
3,LUNG1-002_0001/RTSTRUCT_43245931/GTV.nii.gz,40fcb78dd23c751a7013d4f76682dec3bfa2db13,2025-06-09:20:04:28,1,GTV,LUNG1-002,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2432675512669112458302...,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,...,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)","(100, 99, 26)","(129, 226, 28)","(229, 325, 54)",1.253640e+05,0.0,1.0,0.004308,0.065496,0.004290


In [10]:
import SimpleITK as sitk
import pandas as pd

from pathlib import Path
from damply import dirs
from joblib import Parallel, delayed
from typing import Optional
from itertools import chain

from readii.image_processing import flattenImage, alignImages
from readii.io.loaders import loadImageDatasetConfig
from readii.io.writers.nifti_writer import NIFTIWriter, NiftiWriterIOError
from readii.negative_controls_refactor import NegativeControlManager
from readii.process.config import get_full_data_name
from readii.utils import logger



def get_readii_settings(dataset_config: dict) -> tuple[list, list, list]:
    """Extract READII settings from a configuration dictionary.
    
    Parameters
    ----------
    dataset_config : dict
        Configuration dictionary read in with `loadImageDatasetConfig` containing READII settings
    
    Returns
    -------
    tuple
        A tuple containing:
        - regions: list of regions to process
        - permutations: list of permutations to apply
        - crop: list of crop settings
    """
    readii_config = dataset_config['READII']
    if 'IMAGE_TYPES' not in readii_config:
        message = "READII configuration must contain 'IMAGE_TYPES'."
        logger.error(message)
        raise KeyError(message)
    
    regions = readii_config['IMAGE_TYPES']['regions']

    permutations = readii_config['IMAGE_TYPES']['permutations']

    crop = readii_config['IMAGE_TYPES']['crop']

    return regions, permutations, crop


def get_masked_image_metadata(dataset_index:pd.DataFrame,
                              dataset_config:Optional[dict] = None,
                              image_modality:Optional[str] = None,
                              mask_modality:Optional[str] = None):
    """Get rows of Med-ImageTools index.csv with the mask modality and the corresponding image modality and create a new index with just these rows for READII
    
    Parameters
    ----------
    dataset_index : pd.DataFrame
        DataFrame loaded from a Med-ImageTools index.csv containing image metadata. Must have columns for Modality, ReferencedSeriesUID, and SeriesInstanceUID.
    dataset_config : Optional[dict]
        Dictionary of configuration settings to get image and mask modality from for filtering dataset_index. Must include MIT MODALITIES image and MIT MODALITIES mask. Expected output from running loadImageDatasetConfig.
    image_modality : Optional[str]
        Image modality to filter dataset_index with. Will override dataset_config setting.
    mask_modality : Optional[str]
        Mask modality to filter dataset_index with. Will override dataset_config setting.

    Returns
    -------
    pd.DataFrame
        Subset of the dataset_index with just the masks and their reference images' metadata.
    """

    if image_modality is None:
        if dataset_config is None:
            message = "No image modality setting passed. Must pass a image_modality or dataset_config with an image modality setting."
            logger.error(message)
            raise ValueError(message)
        
        # Get the image modality from config to retrieve from the metadata
        image_modality = dataset_config["MIT"]["MODALITIES"]["image"]
    
    if mask_modality is None:
        if dataset_config is None:
            message = "No mask modality setting passed. Must pass a mask_modality or dataset_config with a mask modality setting."
            logger.error(message)
            raise ValueError(message)
        
        # Get the mask modality from config to retrieve from the metadata
        mask_modality = dataset_config["MIT"]["MODALITIES"]["mask"]

    # Get all metadata rows with the mask modality
    mask_metadata = dataset_index[dataset_index['Modality'] == mask_modality]

    # Get a Series of ReferenceSeriesUIDs from the masks - these point to the images the masks were made on
    referenced_series_ids = mask_metadata['ReferencedSeriesUID']
    
    # Get image metadata rows with a SeriesInstanceUID matching one of the ReferenceSeriesUIDS of the masks
    image_metadata = dataset_index[dataset_index['Modality'] == image_modality]
    masked_image_metadata = image_metadata[image_metadata['SeriesInstanceUID'].isin(referenced_series_ids)]

    # Return the subsetted metadata
    return pd.concat([masked_image_metadata, mask_metadata], sort=True)



def save_out_negative_controls(nifti_writer: NIFTIWriter,
                               patient_id: str,
                               image: sitk.Image,
                               region: str,
                               permutation: str):
    """Save out negative control images using the NIFTIWriter."""

    try:
        out_path = nifti_writer.save(
                        image,
                        PatientID=patient_id,
                        region=region,
                        permutation=permutation
                    )
    except NiftiWriterIOError as e:
        message = f"{permutation} {region} negative control file already exists for {patient_id}. If you wish to overwrite, set overwrite to true in the NIFTIWriter."
        logger.debug(message)
        out_path = None

    return out_path

    

In [18]:
"""Create negative control images and save them out as niftis"""



if dataset is None:
    message = "Dataset name must be provided."
    logger.error(message)
    raise ValueError(message)

config_dir_path = dirs.CONFIG / 'datasets'

dataset_config = loadImageDatasetConfig(dataset, config_dir_path)

dataset_name = dataset_config['DATASET_NAME']
full_data_name = get_full_data_name(config_dir_path / dataset)
logger.info(f"Creating negative controls for dataset: {dataset_name}")

# Extract READII settings
regions, permutations, _crop = get_readii_settings(dataset_config)

# Set up negative control manager with settings from config
manager = NegativeControlManager.from_strings(
    negative_control_types=permutations,
    region_types=regions,
    random_seed=10
)

mit_images_dir_path = dirs.PROCDATA / full_data_name / 'images' /f'mit_{dataset_name}'

dataset_index = pd.read_csv(Path(mit_images_dir_path, f'mit_{dataset_name}_index.csv'))

image_modality = dataset_config["MIT"]["MODALITIES"]["image"]
mask_modality = dataset_config["MIT"]["MODALITIES"]["mask"]

masked_image_index = get_masked_image_metadata(dataset_index = dataset_index,
                                                image_modality = image_modality,
                                                mask_modality = mask_modality)

# StudyInstanceUID
for study, study_data in masked_image_index.groupby('StudyInstanceUID'):
    logger.info(f"Processing StudyInstanceUID: {study}")

    # Get image metadata as a pd.Series
    image_metadata = study_data[study_data['Modality'] == image_modality].squeeze()
    image_path = Path(image_metadata['filepath'])
    # Load in image
    raw_image = sitk.ReadImage(mit_images_dir_path / image_path)
    # Remove extra dimension of image, set origin, spacing, direction to original
    image = alignImages(raw_image, flattenImage(raw_image))

    
    # Get mask metadata as a pd.Series
    all_mask_metadata = study_data[study_data['Modality'] == mask_modality]

    for row_idx, mask_metadata in all_mask_metadata.iterrows():
        readii_sample_id = f"{mask_metadata['PatientID']}_{mask_metadata['ImageID']}"

        mask_path = mit_images_dir_path / Path(mask_metadata['filepath'])
        # Load in mask
        raw_mask = sitk.ReadImage(mask_path)
        mask = alignImages(raw_mask, flattenImage(raw_mask))

        mask_roi_name = mask_metadata['ImageID']
        # Set up writer for saving out the negative controls
    
        nifti_writer = NIFTIWriter(
            root_directory = mit_images_dir_path.parent / f'readii_{dataset_name}' / image_path.parent / mask_roi_name,
            filename_format = f"{image_modality}" + "_{permutation}_{region}.nii.gz",
            overwrite = False,
            create_dirs = True
        )
        
        # Generate each image type and save it out with the nifti writer
        readii_image_paths = Parallel(n_jobs=-1, require="sharedmem")(
                    delayed(save_out_negative_controls)(
                        nifti_writer, 
                        patient_id = image_metadata['PatientID'],
                        image = neg_image,
                        region = region,
                        permutation = permutation
                    ) for neg_image, permutation, region in manager.apply(image, mask)
        )

        
        

In [24]:
# get relative paths to the 

relative_readii_image_paths = [path.relative_to(dirs.PROCDATA) for path in readii_image_paths if path is not None]
relative_mask_path = mask_path.relative_to(dirs.PROCDATA)

In [25]:
pd.DataFrame(data = {'ID': readii_sample_id,
                     'Image': relative_readii_image_paths,
                     'Mask': relative_mask_path})

,ID,Image,Mask
0,LUNG1-001_GTV,TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-...,TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Rad...
1,LUNG1-001_GTV,TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-...,TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Rad...
2,LUNG1-001_GTV,TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-...,TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Rad...
3,LUNG1-001_GTV,TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-...,TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Rad...
4,LUNG1-001_GTV,TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-...,TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Rad...
5,LUNG1-001_GTV,TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-...,TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Rad...
6,LUNG1-001_GTV,TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-...,TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Rad...
7,LUNG1-001_GTV,TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-...,TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Rad...
8,LUNG1-001_GTV,TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-...,TCIA_NSCLC-Radiomics_test/images/mit_NSCLC-Rad...
